In [46]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind
from scipy.stats import pearsonr, spearmanr, mannwhitneyu

# 1 Выгрузка

In [47]:
Dataset = pd.read_excel('D:/SkyPro/Материалы по обучению/!Финальная Аттестация/Финальная первичная аттестация. Вариант 1. Python.xlsx', sheet_name='Dataset')
city_dict = pd.read_excel('D:/SkyPro/Материалы по обучению/!Финальная Аттестация/Финальная первичная аттестация. Вариант 1. Python.xlsx', sheet_name='city_dict')

# 1.1 Изучение данных 

In [48]:
Dataset.head(5)

,id_view,time_view,id_group,id_city,nflag_order
0,10457162393,2023-01-17 13:56:06.118,0,3,1
1,10457152884,2023-01-30 17:38:18.629,1,1,1
2,10457162947,2023-01-07 14:06:22.689,1,1,1
3,10457197478,2023-01-02 11:02:58.189,0,3,0
4,10457176480,2023-01-04 22:56:19.240,0,1,1


In [49]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21939 entries, 0 to 21938
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_view      21939 non-null  int64         
 1   time_view    21939 non-null  datetime64[ns]
 2   id_group     21939 non-null  int64         
 3   id_city      21939 non-null  int64         
 4   nflag_order  21939 non-null  int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 857.1 KB


In [50]:
city_dict.head(5)

,id_city,name_city
0,1,Москва
1,2,Санкт-Петербург
2,3,Новосибирск
3,4,Казань
4,5,Краснодар


# Задание 1.1

Импортируйте файл с данными в Jupyter Notebook и расшифруйте идентификаторы городов с помощью листа city_dict

**Описание логики десйтвий:**
Учитывая что выгрузка файла ексель по книгам была сделана ранее осталось объединить таблицы по индетифактору которому соответсвтует определнный город с помощью .merge()


In [51]:
df = Dataset.merge(city_dict, on = 'id_city', how = 'outer')
df.reset_index()

,index,id_view,time_view,id_group,id_city,nflag_order,name_city
0,0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск
1,1,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск
2,2,10457135372,2023-01-15 10:03:58.011,0,3,0,Новосибирск
3,3,10457162971,2023-01-02 09:39:31.283,0,3,1,Новосибирск
4,4,10457132942,2023-01-23 19:18:03.516,0,3,0,Новосибирск
...,...,...,...,...,...,...,...
21934,21934,10457174004,2023-01-21 11:47:30.578,0,9,1,Воронеж
21935,21935,10457130320,2023-01-24 02:09:49.169,0,9,0,Воронеж
21936,21936,10457166866,2023-01-13 16:22:38.491,1,9,0,Воронеж
21937,21937,10457186695,2023-01-28 02:21:49.007,0,9,1,Воронеж


# Задание 1.2

Рассчитайте результаты АВ-теста и с помощью статистического критерия сделайте вывод

**Описание логики десйтвий:**
Для начала создадим функцию для проведения статистичекого теста. Далее прведем тест сравнив тесовую и рабочию группы между собой!

In [52]:
def test_calc(r1, r2, alpha=.05):
    
    s, p = ttest_ind(r1, r2)
    
    if p<alpha:
        print('Гипотеза H0 не подтверждается: средние не равны')
        print('Разница средних = ', r1.mean(), r2.mean())
    else:
        print('Гипотеза H0  подтверждается: средние равны')
        print('Разница средних = 0')
    return s, p

In [53]:
test_calc(df[df['id_group']==1]['nflag_order'], df[df['id_group']==0]['nflag_order'])

Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.6757851165301899 0.6150932939220395


(9.41595134321716, 5.1365957325220535e-21)

**Вывод:**
Коверсия из просмотра в заказ у группы 1 выше чем у группы ноль на статистически значемую величину. Если принять что группа один является группой с введной фичей для улчшения конверсии, то можно сделать вывод что фича является рабочей - конверсия растет в общем по всеё группе данных! Далее проведем сегментацию для опредления для каких групп введения данной фичи является актуальной а для каких нет!


# Задание 3.3

Запустите цикл, который пробежится по всем городам в датасете. Выявите те города, в которых есть статистически значимая разница между группами А и В. Создайте переменную типа "список", которая будет хранить эти города.

**Описание логики действий:**
Создаем лист уникальных городов all_сities. Цикл идет по созданному листу уникальных городов. Создается датафрейм df_city с фильтром по i-му уникальному городу в датафрейме df. Проводится статистичекйи тест по группам 1 и 0 'id_group' пол конверсии в заказ 'nflag_order' в датафреме df_city в каждом цикле пробежки по данным это будет разный город. 

In [54]:
all_cities = list(df['name_city'].unique())
for i in all_cities:
    print(i)

    df_city = df[df['name_city']==i] 
    test_calc(df_city[df_city['id_group']==1]['nflag_order'], df_city[df_city['id_group']==0]['nflag_order'])
    print()

Новосибирск
Гипотеза H0  подтверждается: средние равны
Разница средних = 0

Москва
Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.691944194419442 0.6172531214528945

Санкт-Петербург
Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.6791930379746836 0.6083916083916084

Казань
Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.6784565916398714 0.596244131455399

Архангельск
Гипотеза H0  подтверждается: средние равны
Разница средних = 0

Челябинск
Гипотеза H0  подтверждается: средние равны
Разница средних = 0

Сочи
Гипотеза H0  подтверждается: средние равны
Разница средних = 0

Хабаровск
Гипотеза H0  подтверждается: средние равны
Разница средних = 0

Нижний Новгород
Гипотеза H0  подтверждается: средние равны
Разница средних = 0

Краснодар
Гипотеза H0  подтверждается: средние равны
Разница средних = 0

Воронеж
Гипотеза H0  подтверждается: средние равны
Разница средних = 0



**Вывод:**
Статистически значимая разница конверсии из просмотра в заказ по тестовым группам есть только в трех городах: Москва, Санкт-Петербург и Казань. В остальных городах  введение фичи статистическуи значимую разницу не дает!

# Задание 3.4

Создайте дополнительную колонку со значением 0 или 1 в зависимости от часа заказа.
1 - если час пик (возьмите, например, 7,8,9,10,17,18,19,20), а 0 - все остальные часы.
Проверьте результат АВ-теста по отдельности на пиковых и не-пиковых заказах и сделайте выводы.

***Описание логики действий***
Создадим дополнительную колонку с определением часа просмотра. Далее создаем колонку для определения явлется ли это время пиковым если 7,8,9,10,17,18,19,20 часов то 1, остальные 0. Создаем цикл пробегаем по значения nflag_pick 0 и 1 для определения статисичеки значимой разницы конверсии для этих двух групп.
Также провели тесты отдельно без циклом проверили верность псотренного цикла!




In [55]:
df['hour']=df['time_view'].dt.hour
df['nflag_pick'] = np.where((df['hour'].isin([7,8,9,10,17,18,19,20])), 1,0)
df.head(5)

,id_view,time_view,id_group,id_city,nflag_order,name_city,hour,nflag_pick
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск,13,0
1,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск,11,0
2,10457135372,2023-01-15 10:03:58.011,0,3,0,Новосибирск,10,1
3,10457162971,2023-01-02 09:39:31.283,0,3,1,Новосибирск,9,1
4,10457132942,2023-01-23 19:18:03.516,0,3,0,Новосибирск,19,1


In [56]:
for i in range(0,2):
    print(i)

    df_pick = df[df['nflag_pick']==i] 
    test_calc(df_pick[df_pick['id_group']==1]['nflag_order'], df_pick[df_pick['id_group']==0]['nflag_order'])
    print()

0
Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.6751420070327292 0.6200055111600992

1
Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.6770637268082818 0.6051008968609866



In [57]:
test_calc(df[(df['id_group']==1) & (df['nflag_pick']==1)]['nflag_order'], df[(df['id_group']==0) & (df['nflag_pick']==1)]['nflag_order'])

Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.6770637268082818 0.6051008968609866


(6.422002058720224, 1.429452278435098e-10)

In [58]:
test_calc(df[(df['id_group']==1) & (df['nflag_pick']==0)]['nflag_order'], df[(df['id_group']==0) & (df['nflag_pick']==0)]['nflag_order'])

Гипотеза H0 не подтверждается: средние не равны
Разница средних =  0.6751420070327292 0.6200055111600992


(6.99724331712408, 2.723734415858154e-12)

**Вывод:**
Для обеих групп имеетсмя статистичеки значимая разница конверсии из просмотра в заказ. Что говорит что для введение данной фичи  положительно влияет для обеих групп!  